In [1]:
from services.url_extractor_service import UrlExtractor

extractor = UrlExtractor.scrape_course_modules_units_and_topics

In [2]:
from langchain_openai import AzureOpenAIEmbeddings
from dotenv import load_dotenv
load_dotenv()
import os

endpoint = "https://pedro-ai.openai.azure.com/"
key = "0d1eed0a4e1b4f169f47038e41317555"
model_name = "text-embedd"

embedings = AzureOpenAIEmbeddings(
    azure_endpoint=endpoint,
    api_key=key,
    azure_deployment=model_name
)

### Get content

In [3]:
api_key = os.getenv('SCRAPPING_TOOL_API_KEY')
url = "https://learn.microsoft.com/en-us/training/courses/ai-102t00"
exam_content = extractor(url, api_key)

In [6]:
import json

exam_modules = json.loads(exam_content)

In [9]:
print(exam_modules)

{'course': 'Course AI-102T00-A: Develop AI solutions in Azure - Training', 'modules': [{'name': 'Develop generative AI apps in Azure - Training', 'url': 'https://learn.microsoft.com/en-us/training/paths/create-custom-copilots-ai-studio', 'units': [{'name': 'Plan and prepare to develop AI solutions on Azure', 'url': 'https://learn.microsoft.com/en-us/training/modules/prepare-azure-ai-development/', 'topics': [{'name': 'Introduction', 'url': 'https://learn.microsoft.com/en-us/training/modules/prepare-azure-ai-development/1-introduction'}, {'name': 'What is AI?', 'url': 'https://learn.microsoft.com/en-us/training/modules/prepare-azure-ai-development/2-what-is-ai'}, {'name': 'Azure AI services', 'url': 'https://learn.microsoft.com/en-us/training/modules/prepare-azure-ai-development/3-azure-ai-services'}, {'name': 'Azure AI Foundry', 'url': 'https://learn.microsoft.com/en-us/training/modules/prepare-azure-ai-development/4-azure-ai-foundry'}, {'name': 'Developer tools and SDKs', 'url': 'http

In [18]:
from pydantic import BaseModel, Field
from typing import List

class Module:
    name: str
    url: str
    numberOfUnits: int

class Unit:
    name: str
    url: str
    numberOfTopics: int

class Topic:
    name: str
    url: str

class ExamContent:
    modules: List[Module]
    units: List[Unit]
    topics: List[Topic]

In [ ]:
def get_exam_module_content(modules):
    

In [17]:
module_1 = exam_modules['modules'][0]

unit_1 = module_1['units'][0]

topic_1 = unit_1['topics'][0]

topic_1

{'name': 'Introduction',
 'url': 'https://learn.microsoft.com/en-us/training/modules/prepare-azure-ai-development/1-introduction'}

In [80]:
from langchain_community.document_loaders import ScrapingAntLoader


def extract_content(module_name, module_url):
    
    scrapingant_loader = ScrapingAntLoader(
        [module_url],  # List of URLs to scrape
        api_key=api_key,  # Get your API key from https://scrapingant.com/
        continue_on_failure=True,  # Ignore unprocessable web pages and log their exceptions
    )
    
    try:
        
        docs = scrapingant_loader.load()
        module_text = docs[0].page_content
        clean_text = module_text.partition(f"# {module_name}")
        content = clean_text[2].partition("## Next unit:")[0]
    except:
        content = "Something went wrong with extraction agent. try again Later"
        
    return content




In [76]:
from langchain_chroma import Chroma
from langchain_core.documents import Document

vector_store = Chroma(
    collection_name="ai-102",
    embedding_function=embedings,
    persist_directory="./chroma_langchain_db"
)

documents = []
number_of_modules = len(exam_modules['modules'])

counter = 0
for k in range(number_of_modules):
    print(f"Module: {k+1}")
    for i in range(len(exam_modules['modules'][k]['units'])):
        print(f"Unit: {i+1}")
        for j in range(len(exam_modules['modules'][k]['units'][i]['topics'])):
            print(f"topic: {j+1}")
            counter += 1
            document = Document(
                id = counter,
                page_content=extract_content(   exam_modules['modules'][k]['units'][i]['topics'][j]['name'],
                                                exam_modules['modules'][k]['units'][i]['topics'][j]['url']),
                metadata = {
                    "course": exam_modules['course'], 
                    'module': exam_modules['modules'][k]['name'], 
                    'unit': exam_modules['modules'][k]['units'][i]['name'],
                    'topic': exam_modules['modules'][k]['units'][i]['topics'][j]['name']
                    },
            )

            documents.append(document)


vector_store.add_documents(documents=documents)


Module: 1
Unit: 1
topic: 1
topic: 2
topic: 3
topic: 4
topic: 5
topic: 6
topic: 7
topic: 8
topic: 9
Unit: 2
topic: 1
topic: 2
topic: 3
topic: 4
topic: 5
topic: 6
topic: 7
Unit: 3
topic: 1
topic: 2
topic: 3
topic: 4
topic: 5
topic: 6
topic: 7
Unit: 4
topic: 1
topic: 2
topic: 3
topic: 4
topic: 5
topic: 6
topic: 7
topic: 8
Unit: 5
topic: 1
topic: 2
topic: 3
topic: 4
topic: 5
topic: 6
topic: 7


Error fetching data from https://learn.microsoft.com/en-us/training/modules/build-copilot-ai-studio/6-knowledge-check, exception: Something went wrong with the server side. Please try later or contact support


IndexError: list index out of range

In [81]:
documents

[Document(id='1', metadata={'course': 'Course AI-102T00-A: Develop AI solutions in Azure - Training', 'module': 'Develop generative AI apps in Azure - Training', 'unit': 'Plan and prepare to develop AI solutions on Azure', 'topic': 'Introduction'}, page_content="\n\nCompleted 100 XP\n\n  * 1 minute\n\nThe growth in the use of artificial intelligence (AI) in general, and\n_generative_ AI in particular means that developers are increasingly required\nto create comprehensive AI solutions. These solutions need to combine machine\nlearning models, AI services, prompt engineering solutions, and custom code.\n\nMicrosoft Azure provides multiple services that you can use to create AI\nsolutions. However, before embarking on an AI application development project,\nit's useful to consider the available options for services, tools, and\nframeworks as well as some principles and practices that can help you succeed.\n\nThis module explores some of the key considerations for planning an AI\ndevelopm

In [18]:
results = vector_store.similarity_search(query = "*", k=1)

In [20]:
for res in results:
    print(res)

page_content='something' metadata={'module': 'prompt engineering', 'source': 'something'}
